# Arbitrage Opportunities Analysis: PSG vs Atletico Madrid

## Objective and Context

This notebook analyzes live betting odds data, extracted from multiple italian betting operators, to explore whether arbitrage opportunities arose during a match.

We chose a PSG vs Atletico Madrid football match (2025/06/16). This is a big match between two strong teams, hopefully resulting in fair quotes, liquidity, and ideal betting market conditions.

**CONTEXT:**

Arbitrage betting involves placing bets on all possible outcomes of an event across different bookmakers to guarantee a profit, regardless of the result.
We explored the two largest Italian markets: [Lottomatica](https://www.lottomatica.it) and [Sisal](https://www.sisal.it).

**KEY GOALS:**

1. Create working scrapers to extract data during a live event;
2. Compare odds across different bookmakers;
3. Detect arbitrage opportunities where total implied probability < 100%
4. Quantify potential profit margins for identified opportunities
5. Visualize odds movements and arbitrage windows over time


**DISCLAIMER:**

This is an academic exercise in the context of a University course. We were informally tasked to download live data regarding a specific event, as a way to practice web scraping techniques. The entire project spanned a single weekend. There is no claim of accuracy nor quality regarding this project.


## Arbitrage Betting Theory

Arbitrage opportunities occur when:
- **Market Inefficiencies:** Different bookmakers have varying assessments of probabilities;
- **Timing Differences:** Odds updates happen at different speeds across platforms;
- **Market Liquidity:** Lower liquidity markets may have less efficient pricing;
- **Information Asymmetry:** Some bookmakers react faster to new information (injuries, weather, etc.).

In the following analysis, we will examine theoretical betting scenarios without considering real-world factors such as rounding errors, transaction fees, interest rates, or inflation. This is a reasonable approximation for live online bets where outcomes are determined within a short timeframe during a single match. Additionally, betting large round amounts (e.g., $1000) minimizes most rounding errors. We assume sufficient liquidity is available and immediate cash rewards are possible.
All odds, when present, are strictly positive.

### Two-Outcome Scenario

For a two-outcome event (_e.g._, a tennis match), arbitrage is possible when: **`1/odds1 + 1/odds2 < 1`**

Where:
- `odds1` = odds for outcome 1 at bookmaker A
- `odds2` = odds for outcome 2 at bookmaker B

**Proof:**

If we bet amounts `p` on `A` and `q` on `B`: 
- the total stake is `S = p + q`
- the possible outcomes are `odds1 * p` or `odds2 * q` 

Events A and B exhaust the probability space. We achieve a deterministic cash payout when `odds1 * p = odds2 * q`. Therefore, when betting an amount `p`, we need:

`q = (odds1/odds2) * p`

The arbitrage opportunity (disregarding discounting and other real-world effects) arises when: `S = p + q < odds1 * p (= odds2 * q)`. Substituting the relationship between `q` and `p`, we obtain the arbitrage condition above.


### Three-Outcome Scenario

For a three-outcome event (e.g., football match: Win/Draw/Loss), arbitrage is possible when: **`1/odds1 + 1/odds2 + 1/odds3 < 1`**

Where:
- `odds1` = odds for outcome 1 (e.g., Home Win)
- `odds2` = odds for outcome 2 (e.g., Draw)
- `odds3` = odds for outcome 3 (e.g., Away Win)

The proof is identical to the two-outcome scenario.

Finally, the **arbitrage ratio** can be extrapolated as: `(1/odds1 + 1/odds2 + ... + 1/oddsn)`

- If < 1: Arbitrage opportunity exists
- If = 1: Break-even (no profit, no loss)
- If > 1: No arbitrage possible (bookmaker edge)

## Analysis

In [1]:
import pandas as pd
import numpy as np


def df_disp(df: pd.DataFrame):
    display(df.info())
    display(df.describe())
    display(df.head())

### Lottomatica Data Cleanup

In [2]:
lottomatica_raw = pd.read_csv("../data/lottomatica_scraper_20250615_210109.csv", sep=",", index_col=0, parse_dates=True)

df_disp(lottomatica_raw)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69 entries, 2025-06-15 21:01:55.166929 to 2025-06-15 23:03:05.735462
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   source                69 non-null     object 
 1   match_id              69 non-null     object 
 2   home_team             69 non-null     object 
 3   away_team             69 non-null     object 
 4   home_win              57 non-null     float64
 5   draw                  67 non-null     float64
 6   away_win              67 non-null     float64
 7   home_or_draw          0 non-null      float64
 8   away_or_draw          0 non-null      float64
 9   home_or_away          0 non-null      float64
 10  over_1_5              0 non-null      float64
 11  under_1_5             0 non-null      float64
 12  over_2_5              0 non-null      float64
 13  under_2_5             0 non-null      float64
 14  over_3_5              0 

None

,home_win,draw,away_win,home_or_draw,away_or_draw,home_or_away,over_1_5,under_1_5,over_2_5,under_2_5,over_3_5,under_3_5,both_teams_score_yes,both_teams_score_no
count,57.000000,67.000000,67.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.349825,8.981343,54.183582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.350431,7.978654,71.125114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.010000,3.300000,3.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.050000,4.475000,7.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.300000,4.650000,10.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.350000,10.500000,55.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2.000000,50.000000,225.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,source,match_id,home_team,away_team,home_win,draw,away_win,home_or_draw,away_or_draw,home_or_away,over_1_5,under_1_5,over_2_5,under_2_5,over_3_5,under_3_5,both_teams_score_yes,both_teams_score_no
timestamp,,,,,,,,,,,,,,,,,,
2025-06-15 21:01:55.166929,Lottomatica,psg-atletico-madrid,PSG,Atletico Madrid,1.95,3.50,3.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-15 21:02:40.413956,Lottomatica,psg-atletico-madrid,PSG,Atletico Madrid,2.00,3.55,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-15 21:04:13.500877,Lottomatica,psg-atletico-madrid,PSG,Atletico Madrid,2.00,3.50,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-15 21:05:31.916741,Lottomatica,psg-atletico-madrid,PSG,Atletico Madrid,2.00,3.50,3.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-15 21:07:42.582592,Lottomatica,psg-atletico-madrid,PSG,Atletico Madrid,2.00,3.50,3.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Unfortunately, due to a technical error, only 1X2 single quotes were scraped from the Lottomatica webpage. Let's clean up the data:

In [3]:
lotto = lottomatica_raw[["home_win", "away_win", "draw"]].copy()
lotto = lotto.dropna(axis=0, how="all")
lotto = lotto.sort_index().reset_index()

#lotto["start"] = pd.to_datetime(lotto["timestamp"])
# Create an end timestamp by shifting the start time up by one row
#lotto["end"] = lotto.iloc[1:,0].reset_index(drop=True).copy()
# and filling the last row with a default value of 5 minutes after the start time)
#lotto["end"] = lotto["end"].fillna(lotto["start"] + pd.Timedelta(minutes=5))

lotto.columns = ["lott_" + col for col in lotto.columns]

df_disp(lotto)
lotto


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   lott_timestamp  67 non-null     datetime64[ns]
 1   lott_home_win   57 non-null     float64       
 2   lott_away_win   67 non-null     float64       
 3   lott_draw       67 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 2.2 KB


None

,lott_timestamp,lott_home_win,lott_away_win,lott_draw
count,67,57.000000,67.000000,67.000000
mean,2025-06-15 21:58:21.954742272,1.349825,54.183582,8.981343
min,2025-06-15 21:01:55.166929,1.010000,3.300000,3.300000
25%,2025-06-15 21:29:01.446328064,1.050000,7.750000,4.475000
50%,2025-06-15 21:53:18.025061120,1.300000,10.500000,4.650000
75%,2025-06-15 22:26:48.442430464,1.350000,55.000000,10.500000
max,2025-06-15 23:02:53.399668,2.000000,225.000000,50.000000
std,NaN,0.350431,71.125114,7.978654


,lott_timestamp,lott_home_win,lott_away_win,lott_draw
0,2025-06-15 21:01:55.166929,1.95,3.35,3.50
1,2025-06-15 21:02:40.413956,2.00,3.30,3.55
2,2025-06-15 21:04:13.500877,2.00,3.30,3.50
3,2025-06-15 21:05:31.916741,2.00,3.35,3.50
4,2025-06-15 21:07:42.582592,2.00,3.35,3.50


,lott_timestamp,lott_home_win,lott_away_win,lott_draw
0,2025-06-15 21:01:55.166929,1.95,3.35,3.50
1,2025-06-15 21:02:40.413956,2.00,3.30,3.55
2,2025-06-15 21:04:13.500877,2.00,3.30,3.50
3,2025-06-15 21:05:31.916741,2.00,3.35,3.50
4,2025-06-15 21:07:42.582592,2.00,3.35,3.50
...,...,...,...,...
62,2025-06-15 22:59:19.329834,NaN,200.00,9.25
63,2025-06-15 22:59:50.977301,NaN,200.00,9.25
64,2025-06-15 23:01:19.893336,NaN,200.00,9.25
65,2025-06-15 23:02:07.187629,NaN,200.00,9.25


Regarding **data validation**, all simple fundamental checks are visually performed and passed:

- a reasonable amount of missing quotes is tolerated and expected;
- all odds are strictly positive real numbers;
- all rows have at a least one valid quote.

### Sisal Data Cleanup

In [4]:
sisal_raw = pd.read_csv("../data/sisal_scraper_20250615_210219.csv", sep=",", index_col=0, parse_dates=True)

df_disp(sisal_raw)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 692 entries, 2025-06-15 21:02:29.681026 to 2025-06-15 23:02:59.692660
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   source                692 non-null    object 
 1   match_id              692 non-null    object 
 2   home_team             692 non-null    object 
 3   away_team             692 non-null    object 
 4   home_win              561 non-null    float64
 5   draw                  632 non-null    float64
 6   away_win              632 non-null    float64
 7   home_or_draw          262 non-null    float64
 8   away_or_draw          633 non-null    float64
 9   home_or_away          432 non-null    float64
 10  over_1_5              0 non-null      float64
 11  under_1_5             0 non-null      float64
 12  over_2_5              632 non-null    float64
 13  under_2_5             595 non-null    float64
 14  over_3_5              6

None

,home_win,draw,away_win,home_or_draw,away_or_draw,home_or_away,over_1_5,under_1_5,over_2_5,under_2_5,over_3_5,under_3_5,both_teams_score_yes,both_teams_score_no
count,561.000000,632.000000,632.000000,262.000000,633.000000,432.000000,0.0,0.0,632.000000,595.000000,632.000000,631.000000,632.000000,621.000000
mean,1.312103,36.234019,77.268354,1.117214,11.651311,1.122963,NaN,NaN,2.788244,2.899731,3.004446,1.476735,2.670823,1.876683
std,0.352196,93.736794,134.381839,0.103610,16.063690,0.095342,NaN,NaN,5.620623,1.108587,1.501635,0.277548,2.770995,0.398317
min,1.010000,3.250000,3.400000,1.010000,1.670000,1.010000,NaN,NaN,1.140000,1.520000,1.700000,1.020000,1.450000,1.010000
25%,1.050000,4.500000,8.500000,1.020000,2.910000,1.020000,NaN,NaN,1.200000,2.000000,1.950000,1.300000,1.620000,1.570000
50%,1.070000,11.000000,30.000000,1.040000,7.440000,1.140000,NaN,NaN,1.520000,2.500000,2.750000,1.380000,1.720000,2.000000
75%,1.380000,16.000000,60.000000,1.220000,10.890000,1.240000,NaN,NaN,1.750000,4.250000,3.137500,1.720000,2.250000,2.150000
max,2.050000,501.000000,501.000000,1.260000,68.980000,1.290000,NaN,NaN,33.000000,5.000000,12.000000,2.000000,22.000000,2.500000


,source,match_id,home_team,away_team,home_win,draw,away_win,home_or_draw,away_or_draw,home_or_away,over_1_5,under_1_5,over_2_5,under_2_5,over_3_5,under_3_5,both_teams_score_yes,both_teams_score_no
timestamp,,,,,,,,,,,,,,,,,,
2025-06-15 21:02:29.681026,Sisal,paris-saint-germain-atletico-madrid,Paris Saint-Germain,Atletico Madrid,2.0,3.4,3.6,1.24,1.71,1.26,NaN,NaN,1.7,2.1,2.9,1.36,1.6,2.2
2025-06-15 21:02:39.872720,Sisal,paris-saint-germain-atletico-madrid,Paris Saint-Germain,Atletico Madrid,2.0,3.4,3.6,1.24,1.71,1.26,NaN,NaN,1.7,2.1,2.9,1.36,1.6,2.2
2025-06-15 21:02:50.054884,Sisal,paris-saint-germain-atletico-madrid,Paris Saint-Germain,Atletico Madrid,2.0,3.4,3.6,1.24,1.71,1.26,NaN,NaN,1.7,2.1,2.9,1.36,1.6,2.2
2025-06-15 21:03:02.028270,Sisal,paris-saint-germain-atletico-madrid,Paris Saint-Germain,Atletico Madrid,2.0,3.4,3.6,1.24,1.71,1.26,NaN,NaN,1.7,2.1,2.9,1.36,1.6,2.2
2025-06-15 21:03:12.275042,Sisal,paris-saint-germain-atletico-madrid,Paris Saint-Germain,Atletico Madrid,2.0,3.4,3.6,1.24,1.71,1.26,NaN,NaN,1.7,2.1,2.9,1.36,1.6,2.2


This scraper worked as expected. However, due to only having 1X2 quotes from Lottomatica, we will have to restrict our analysis to single quotes and double chance quotes from Sisal:

In [6]:
sisal = sisal_raw.iloc[:,4:10].copy()
sisal = sisal.dropna(axis=0, how="all")
sisal = sisal.sort_index().reset_index()
sisal['timestamp'] = pd.to_datetime(sisal['timestamp'])
sisal.columns = ["sis_" + col for col in sisal.columns]

df_disp(sisal)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   sis_timestamp     633 non-null    datetime64[ns]
 1   sis_home_win      561 non-null    float64       
 2   sis_draw          632 non-null    float64       
 3   sis_away_win      632 non-null    float64       
 4   sis_home_or_draw  262 non-null    float64       
 5   sis_away_or_draw  633 non-null    float64       
 6   sis_home_or_away  432 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 34.7 KB


None

,sis_timestamp,sis_home_win,sis_draw,sis_away_win,sis_home_or_draw,sis_away_or_draw,sis_home_or_away
count,633,561.000000,632.000000,632.000000,262.000000,633.000000,432.000000
mean,2025-06-15 22:01:12.397410816,1.312103,36.234019,77.268354,1.117214,11.651311,1.122963
min,2025-06-15 21:02:29.681026,1.010000,3.250000,3.400000,1.010000,1.670000,1.010000
25%,2025-06-15 21:32:18.294703104,1.050000,4.500000,8.500000,1.020000,2.910000,1.020000
50%,2025-06-15 22:01:15.584935936,1.070000,11.000000,30.000000,1.040000,7.440000,1.140000
75%,2025-06-15 22:31:51.006784,1.380000,16.000000,60.000000,1.220000,10.890000,1.240000
max,2025-06-15 23:00:15.596895,2.050000,501.000000,501.000000,1.260000,68.980000,1.290000
std,NaN,0.352196,93.736794,134.381839,0.103610,16.063690,0.095342


,sis_timestamp,sis_home_win,sis_draw,sis_away_win,sis_home_or_draw,sis_away_or_draw,sis_home_or_away
0,2025-06-15 21:02:29.681026,2.0,3.4,3.6,1.24,1.71,1.26
1,2025-06-15 21:02:39.872720,2.0,3.4,3.6,1.24,1.71,1.26
2,2025-06-15 21:02:50.054884,2.0,3.4,3.6,1.24,1.71,1.26
3,2025-06-15 21:03:02.028270,2.0,3.4,3.6,1.24,1.71,1.26
4,2025-06-15 21:03:12.275042,2.0,3.4,3.6,1.24,1.71,1.26


### Arbitrage Check

The Lottomatica scraper performed significantly slower than expected (this issue has since been fixed), therefore we cannot check if the intended sampling frequency (10 seconds) was sufficient to capture every quote update. We are going to merge the available data and assume that quotes remained constant during the sampling period. 

In [8]:
# Use merge_asof for time-based joining
merged_odds = pd.merge_asof(
    sisal,
    lotto,
    left_on='sis_timestamp',
    right_on='lott_timestamp',
    direction='backward'
)

merged_odds.head()

,sis_timestamp,sis_home_win,sis_draw,sis_away_win,sis_home_or_draw,sis_away_or_draw,sis_home_or_away,lott_timestamp,lott_home_win,lott_away_win,lott_draw
0,2025-06-15 21:02:29.681026,2.0,3.4,3.6,1.24,1.71,1.26,2025-06-15 21:01:55.166929,1.95,3.35,3.50
1,2025-06-15 21:02:39.872720,2.0,3.4,3.6,1.24,1.71,1.26,2025-06-15 21:01:55.166929,1.95,3.35,3.50
2,2025-06-15 21:02:50.054884,2.0,3.4,3.6,1.24,1.71,1.26,2025-06-15 21:02:40.413956,2.00,3.30,3.55
3,2025-06-15 21:03:02.028270,2.0,3.4,3.6,1.24,1.71,1.26,2025-06-15 21:02:40.413956,2.00,3.30,3.55
4,2025-06-15 21:03:12.275042,2.0,3.4,3.6,1.24,1.71,1.26,2025-06-15 21:02:40.413956,2.00,3.30,3.55


In [43]:
def eval_arbitrage_ratio(s1: pd.Series, s2: pd.Series, s3: pd.Series|None = None) -> pd.Series:
    """
    Calculate the arbitrage ratio between betting odds series.
    The formula is: 1/s1 + 1/s2 + 1/s3 (if s3 is provided).
    """
    print(f"Calculating arbitrage ratio for series: {s1.name}, {s2.name}, {s3.name if s3 is not None else 'None'}")
    return 1 / s1 + 1 / s2 + 0 if s3 is None else 1 / s3

In [45]:
arbitrageable_odds = [
    # Single odds (A, B, B)
    ['sis_home_win', 'lott_away_win', 'lott_draw'],
    ['sis_draw', 'lott_home_win', 'lott_away_win'],
    ['sis_away_win', 'lott_home_win', 'lott_draw'],
    # Single odds (A, A, B)
    ['sis_home_win', 'sis_draw', 'lott_away_win'],
    ['sis_home_win', 'sis_away_win', 'lott_draw'],
    ['sis_away_win', 'sis_draw', 'lott_home_win'],
    # Double chance
    ['lott_home_win', 'sis_away_or_draw'],
    ['lott_draw', 'sis_home_or_away'],
    ['lott_away_win', 'sis_home_or_draw']
]

arbitrage_ratios = merged_odds[["lott_timestamp", "sis_timestamp"]].copy()
for odds in arbitrageable_odds:
    series = [merged_odds[col] for col in odds]
    arbitrage_ratios[f'{"_".join(odds)}'] = eval_arbitrage_ratio(*series)

arbitrage_ratios.head()
                                                                

Calculating arbitrage ratio for series: sis_home_win, lott_away_win, lott_draw
Calculating arbitrage ratio for series: sis_draw, lott_home_win, lott_away_win
Calculating arbitrage ratio for series: sis_away_win, lott_home_win, lott_draw
Calculating arbitrage ratio for series: sis_home_win, sis_draw, lott_away_win
Calculating arbitrage ratio for series: sis_home_win, sis_away_win, lott_draw
Calculating arbitrage ratio for series: sis_away_win, sis_draw, lott_home_win
Calculating arbitrage ratio for series: lott_home_win, sis_away_or_draw, None
Calculating arbitrage ratio for series: lott_draw, sis_home_or_away, None
Calculating arbitrage ratio for series: lott_away_win, sis_home_or_draw, None


,lott_timestamp,sis_timestamp,sis_home_win_lott_away_win_lott_draw,sis_draw_lott_home_win_lott_away_win,sis_away_win_lott_home_win_lott_draw,sis_home_win_sis_draw_lott_away_win,sis_home_win_sis_away_win_lott_draw,sis_away_win_sis_draw_lott_home_win,lott_home_win_sis_away_or_draw,lott_draw_sis_home_or_away,lott_away_win_sis_home_or_draw
0,2025-06-15 21:01:55.166929,2025-06-15 21:02:29.681026,0.285714,0.298507,0.285714,0.298507,0.285714,0.512821,1.097616,1.079365,1.104959
1,2025-06-15 21:01:55.166929,2025-06-15 21:02:39.872720,0.285714,0.298507,0.285714,0.298507,0.285714,0.512821,1.097616,1.079365,1.104959
2,2025-06-15 21:02:40.413956,2025-06-15 21:02:50.054884,0.281690,0.303030,0.281690,0.303030,0.281690,0.500000,1.084795,1.075341,1.109482
3,2025-06-15 21:02:40.413956,2025-06-15 21:03:02.028270,0.281690,0.303030,0.281690,0.303030,0.281690,0.500000,1.084795,1.075341,1.109482
4,2025-06-15 21:02:40.413956,2025-06-15 21:03:12.275042,0.281690,0.303030,0.281690,0.303030,0.281690,0.500000,1.084795,1.075341,1.109482


Extracting arbitrage opportunities:

In [ ]:
arbitrage_ratios = arbitrage_ratios.dropna(axis=0, how="all"
                        
arbitr
arbitrage_ratios = arbitrage_ratios.set_index("lott_timestamp")     

NameError: name 'arbitr' is not defined